In [ ]:
!pip install openmeteo-requests
!pip install requests-cache retry-requests numpy pandas

In [5]:
from pyowm import OWM
from pyowm.utils import config
from pyowm.utils import timestamps

# ---------- FREE API KEY examples ---------------------

owm = OWM('c84d29851bb6e85ff1b4517a18ac4b8a')
mgr = owm.weather_manager()


# Search for current weather in London (Great Britain) and get details
observation = mgr.weather_at_place('Orange')
w = observation.weather

w.detailed_status         # 'clouds'
w.wind()                  # {'speed': 4.6, 'deg': 330}
w.humidity                # 87
w.temperature('fahrenheit')  # {'temp_max': 10.5, 'temp': 9.7, 'temp_min': 9.0}
w.rain                    # {}
w.heat_index              # None
w.clouds                  # 75

# Will it be clear tomorrow at this time in Milan (Italy) ?
forecast = mgr.forecast_at_place('Orange', 'daily')
answer = forecast.will_be_clear_at(timestamps.tomorrow())

# ---------- PAID API KEY example ---------------------

config_dict = config.get_default_config_for_subscription_type('professional')
owm = OWM('c84d29851bb6e85ff1b4517a18ac4b8a', config_dict)

# What's the current humidity in Berlin (Germany) ?
one_call_object = mgr.one_call(lat=32.2343321, lon=-117.4105)
one_call_object.current.humidity

UnauthorizedError: Invalid API Key provided

In [6]:
from pyowm.owm import OWM
from pyowm.utils.config import get_default_config_for_subscription_type
config_dict = get_default_config_for_subscription_type('professional')
owm = OWM('c84d29851bb6e85ff1b4517a18ac4b8a', config_dict)

In [7]:
mgr = owm.weather_manager()
observation = mgr.weather_at_place('Orange, CA')
observation.weather.detailed_status  # Nuageux

UnauthorizedError: Invalid API Key provided

In [9]:
import openmeteo_requests

import pandas as pd
import requests_cache
from retry_requests import retry
import numpy as np
import math

# Temperature and wind helpers
def fahrenheit_to_kelvin(f):
    return ((f - 32) * 5) / 9 + 273.15

def kelvin_to_fahrenheit(k):
    return (k - 273.15) * 9 / 5 + 32

def kelvin_to_celsius(k):
    return k - 273.15

def celsius_to_kelvin(c):
    return c + 273.15

# Bröde et al. (2012) logarithmic wind scaling
def scale_windspeed(va, h):
    c = 1 / math.log10(10 / 0.01)
    return va * math.log10(h / 0.01) * c

# Li and Chan (2006) Normal Effective Temperature
def calculate_normal_effective_temperature(t2_k, va, rh):
    t2_k = kelvin_to_celsius(t2_k)
    v = scale_windspeed(va, 1.2)
    ditermeq = 1 / (1.76 + 1.4 * v**0.75)
    net = 37 - ((37 - t2_k) / (0.68 - 0.0014 * rh + ditermeq)) - 0.29 * t2_k * (1 - 0.01 * rh)
    return celsius_to_kelvin(net)

# Apparent Temperature (Australian BoM)
def calculate_apparent_temperature(t2_k, va, rh):
    t2_c = kelvin_to_celsius(t2_k)
    e = (rh / 100) * 6.105 * math.exp(17.27 * t2_c / (237.7 + t2_c))
    at_c = t2_c + 0.33 * e - 0.70 * va - 4.00
    return celsius_to_kelvin(at_c)


# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": 33.3453454,
	"longitude": -117.24352353,
	"hourly": ["temperature_2m", "relative_humidity_2m", "wind_gusts_10m", "wind_speed_10m", "precipitation"],
	"timezone": "America/Los_Angeles",
	"wind_speed_unit": "mph",
	"temperature_unit": "fahrenheit",
	"precipitation_unit": "inch",
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates: {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation: {response.Elevation()} m asl")
print(f"Timezone: {response.Timezone()}{response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0: {response.UtcOffsetSeconds()}s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
hourly_wind_gusts_10m = hourly.Variables(2).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(3).ValuesAsNumpy().round(1)
hourly_precipitation = hourly.Variables(4).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end =  pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
hourly_data["wind_gusts_10m"] = hourly_wind_gusts_10m
hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
hourly_data["precipitation"] = hourly_precipitation

hourly_dataframe = pd.DataFrame(data = hourly_data)

# Compute NET (E.T.) and Apparent Temperature for each row
hourly_dataframe['effective_temp'] = hourly_dataframe.apply(
    lambda r: kelvin_to_fahrenheit(calculate_normal_effective_temperature(
        fahrenheit_to_kelvin(r['temperature_2m']),
        r['wind_speed_10m'] * 0.44704,
        r['relative_humidity_2m']
    )),
    axis=1
)
hourly_dataframe['apparent_temp'] = hourly_dataframe.apply(
    lambda r: kelvin_to_fahrenheit(calculate_apparent_temperature(
        fahrenheit_to_kelvin(r['temperature_2m']),
        r['wind_speed_10m'] * 0.44704,
        r['relative_humidity_2m']
    )),
    axis=1
)

for _, row in hourly_dataframe.iterrows():
    et = row['effective_temp']
    at = row['apparent_temp']
    print(f"Date: {row['date']}, NET: {et:.2f} °F, AT: {at:.2f} °F, Temp: {row['temperature_2m']} °F, Wind: {row['wind_speed_10m']} mph, RH: {row['relative_humidity_2m']} %")
print("\nHourly data\n", hourly_dataframe)


Coordinates: 33.33513641357422°N -117.24481201171875°E
Elevation: 173.0 m asl
Timezone: b'America/Los_Angeles'b'GMT-8'
Timezone difference to GMT+0: -28800s

Hourly data
                          date  temperature_2m  relative_humidity_2m  \
0   2026-01-11 08:00:00+00:00       53.353401                  25.0   
1   2026-01-11 09:00:00+00:00       53.263401                  23.0   
2   2026-01-11 10:00:00+00:00       53.533398                  22.0   
3   2026-01-11 11:00:00+00:00       53.533398                  23.0   
4   2026-01-11 12:00:00+00:00       52.813400                  22.0   
..                        ...             ...                   ...   
163 2026-01-18 03:00:00+00:00       55.507999                  16.0   
164 2026-01-18 04:00:00+00:00       54.158001                  16.0   
165 2026-01-18 05:00:00+00:00       53.888000                  17.0   
166 2026-01-18 06:00:00+00:00       53.618000                  17.0   
167 2026-01-18 07:00:00+00:00       52.987999   

In [11]:
hourly_dataframe

,date,temperature_2m,relative_humidity_2m,wind_gusts_10m,wind_speed_10m,precipitation
0,2026-01-10 08:00:00+00:00,52.633400,19.0,21.698900,16.482607,0.0
1,2026-01-10 09:00:00+00:00,50.203400,22.0,14.540500,11.249686,0.0
2,2026-01-10 10:00:00+00:00,47.953400,30.0,11.856100,6.891267,0.0
3,2026-01-10 11:00:00+00:00,47.233398,26.0,10.737600,5.663631,0.0
4,2026-01-10 12:00:00+00:00,49.213402,26.0,16.330101,9.461742,0.0
...,...,...,...,...,...,...
163,2026-01-17 03:00:00+00:00,59.917999,18.0,3.131800,5.556592,0.0
164,2026-01-17 04:00:00+00:00,58.298000,17.0,4.697700,6.967096,0.0
165,2026-01-17 05:00:00+00:00,58.028000,17.0,7.158400,8.249648,0.0
166,2026-01-17 06:00:00+00:00,57.848000,17.0,9.171700,9.261288,0.0


In [7]:
import numpy as np



# convert Celsius to Kelvin
def celsius_to_kelvin(tc):
    tk = tc + 273.15
    return tk


# convert Kelvin to Celsius
def kelvin_to_celsius(tk):
    tc = tk - 273.15
    return tc


# convert Kelvin to Fahrenheit
def kelvin_to_fahrenheit(tk):
    tf = (tk - 273.15) * 9 / 5 + 32
    return tf


# convert Fahrenheit to Celsius
def fahrenheit_to_celsius(tf):
    tc = (tf - 32) * 5 / 9
    return tc


# convert Fahrenheit to Kelvin
def fahrenheit_to_kelvin(tf):
    tk = (tf + 459.67) * 5 / 9
    return tk


def scale_windspeed(va, h):
    """
    Scaling wind speed from 10 metres to height h
        :param va: (float array) 10m wind speed [m/s]
        :param h: (float array) height at which wind speed needs to be scaled [m]
        returns wind speed at height h
    Reference: Bröde et al. (2012)
    https://doi.org/10.1007/s00484-011-0454-1
    """
    c = 1 / np.log10(10 / 0.01)  #
    c = 0.333333333333
    vh = va * np.log10(h / 0.01) * c

    return vh



def calculate_normal_effective_temperature(t2_k, va, rh):
    """
    NET - Normal Effective Temperature
        :param t2_k: (float array) 2m temperature [K]
        :param va: (float array) wind speed at 10 meters [m/s]
        :param rh: (float array) relative humidity percentage [%]
        returns normal effective temperature [K]
    Reference: Li and Chan (2006)
    https://doi.org/10.1017/S1350482700001602
    """
    t2_k = kelvin_to_celsius(t2_k)
    v = scale_windspeed(va, 1.2)  # formula requires wind speed at 1.2m
    ditermeq = 1 / (1.76 + 1.4 * v**0.75)
    net = (
        37
        - ((37 - t2_k) / (0.68 - 0.0014 * rh + ditermeq))
        - 0.29 * t2_k * (1 - 0.01 * rh)
    )
    net_k = celsius_to_kelvin(net)

    return net_k




def calculate_nonsaturation_vapour_pressure(t2_k, rh):
    """
    Non-saturation vapour pressure using temperature and relative humidity
        :param t2_k: (float) temperature [K]
        :param rh: (float) relative humidity percentage [%]
        returns vapour pressure [hPa]
    Reference: Steadman (1984)
    """
    t2_c = kelvin_to_celsius(t2_k)
    e = (rh / 100) * 6.105 * np.exp(17.27 * t2_c / (237.7 + t2_c))
    return e

def calculate_apparent_temperature(t2_k, va, rh):
    """
    Apparent Temperature - version without radiation
        :param t2_k: (float array) 2m temperature [K]
        :param va: (float array) wind speed at 10 meters [m/s]
        :param rh: (float array) relative humidity percentage [%]
        returns apparent temperature [K]
    Reference: Steadman (1984)
    https://doi.org/10.1175/1520-0450(1984)023%3C1674:AUSOAT%3E2.0.CO;2
    See also: http://www.bom.gov.au/info/thermal_stress/#atapproximation
    """
    t2_c = kelvin_to_celsius(t2_k)
    e = calculate_nonsaturation_vapour_pressure(t2_k, rh)
    at = t2_c + 0.33 * e - 0.7 * va - 4
    at_k = celsius_to_kelvin(at)

    return at_k


In [ ]:
et_k = calculate_normal_effective_temperature(fahrenheit_to_kelvin(90), 0.0, 1)
et = kelvin_to_fahrenheit(et_k)

75.05044140696785

In [10]:
for index, row in hourly_dataframe.iterrows():

    et = kelvin_to_fahrenheit(calculate_normal_effective_temperature(fahrenheit_to_kelvin(row['temperature_2m']), row['wind_speed_10m'], row['relative_humidity_2m']))
    at = kelvin_to_fahrenheit(calculate_apparent_temperature(fahrenheit_to_kelvin(row['temperature_2m']), row['wind_speed_10m'], row['relative_humidity_2m']))
    print(f"Date: {row['date']}, NET: {et:.2f} °F, AT: {at:.2f} °F, Temp: {row['temperature_2m']} °F, Wind: {row['wind_speed_10m']} mph, RH: {row['relative_humidity_2m']} %")

TypeError: 'int' object is not subscriptable